In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [ ]:
import os
import tensorflow as tf
import keras
import cv2
import numpy as np
from enum import Enum
%pylab inline

In [2]:
class CameraLabel(Enum):
    HTC_1_M7 = 0
    LG_Nexus_5x = 1
    Motorola_Droid_Maxx = 2
    Motorola_Nexus_6 = 3
    Motorola_X = 4
    Samsung_Galaxy_Note3 = 5
    Samsung_Galaxy_S4 = 6
    Sony_NEX_7 = 7
    iPhone_4s = 8
    iPhone_6 = 9

In [3]:
def load(path='data-224', load_ratio=1.0, verbose=False):
    print('Loading images')
    data = []
    labels = []
    # Read data from every type of generated data
    for data_path in os.listdir(path):
        full_data_path = os.path.join(path, data_path)
        # Read data from every camera type
        for label in CameraLabel:
            camera_path = os.path.join(full_data_path, label.name.replace('_', '-'))
            if verbose:
                print("Loading images for ", camera_path)
            filenames = os.listdir(camera_path)
            last_index = int(load_ratio * len(filenames))
            for filename in filenames[:last_index]:
                data += [cv2.imread(os.path.join(camera_path, filename))]
                labels += [CameraLabel[label.name].value]
    return np.asarray(data), np.expand_dims(np.asarray(labels), axis=1)

def load_validation(path='data/validation'):
    data = []
    labels = []
    # Read data from every camera type
    for label in CameraLabel:
        camera_path = os.path.join(path, label.name.replace('_', '-'))
        print("Loading images for ", camera_path)
        for filename in os.listdir(camera_path):
            img = cv2.imread(os.path.join(camera_path, filename))
            if img.shape[0] != 224 or img.shape[1] != 224 or img.shape[2] != 3:
                print('Loaded an image with improper dimensions. Discarding')
            else:
                data += [img]
                labels += [CameraLabel[label.name].value]
    return np.asarray(data), np.expand_dims(np.asarray(labels), axis=1)

In [4]:
valid_images, valid_labels = load_validation()

Loading images for  data/validation/HTC-1-M7
Loading images for  data/validation/LG-Nexus-5x
Loading images for  data/validation/Motorola-Droid-Maxx
Loading images for  data/validation/Motorola-Nexus-6
Loaded an image with improper dimensions. Discarding
Loading images for  data/validation/Motorola-X
Loading images for  data/validation/Samsung-Galaxy-Note3
Loading images for  data/validation/Samsung-Galaxy-S4
Loading images for  data/validation/Sony-NEX-7
Loading images for  data/validation/iPhone-4s
Loading images for  data/validation/iPhone-6


In [6]:
#images, labels = load(verbose=True)
images, labels = load(path='data-512', load_ratio=0.5, verbose=True)

In [5]:
""" Good ol' tandem shuffle. """
def shuffle(a, b, c=None):
    assert len(a) == len(b)
    state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(state)
    np.random.shuffle(b)
    if c is not None:
        assert len(c) == len(b)
        np.random.set_state(state)
        np.random.shuffle(c)
  
'''
Crops a randomly-selected 224x224x3 image from a larger image
'''
def crop_224(images, crops_per_image=1):
    out_images = []
    for image in images:
        row_index = np.random.randint(0, image.shape[0] - 224)
        col_index = np.random.randint(0, image.shape[1] - 224)
        out_images += [image[row_index:row_index + 224, col_index:col_index + 224, :]]
    return np.asarray(out_images)

def center_crop(img, crop_dim=224):
    edge = crop_dim // 2
    height, width, _ = img.shape
    center_height = height // 2
    center_width = width // 2
    top = center_height - edge
    bottom = center_height + edge
    left = center_width - edge
    right = center_width + edge
    return img[top:bottom,left:right]


In [9]:
'''
# Convert labels to one-hot (using tf for fun):
one_hot_graph = tf.Graph()
one_hot_session = tf.Session(graph=one_hot_graph)
with one_hot_graph.as_default():
    label_input = tf.placeholder(tf.int32)
    one_hot = tf.one_hot(label_input, 10)
label_one_hot = np.asarray(one_hot.eval(feed_dict={label_input: labels}, session=one_hot_session))
#########
'''
label_one_hot = tf.keras.utils.to_categorical(labels, num_classes=10)
valid_label_one_hot = tf.keras.utils.to_categorical(valid_labels, num_classes=10)
validation_ratio = int(len(images) * 0.8)
#shuffle(images, label_one_hot)  # Shuffle before dividing into training/validation sets
'''
train_data, train_labels = images[:validation_ratio], label_one_hot[:validation_ratio]
valid_data, valid_labels = images[validation_ratio:], label_one_hot[validation_ratio:]
training_samples = len(train_data)
print(train_data.shape, train_labels.shape)
print(valid_data.shape, valid_labels.shape)
'''

'\ntrain_data, train_labels = images[:validation_ratio], label_one_hot[:validation_ratio]\nvalid_data, valid_labels = images[validation_ratio:], label_one_hot[validation_ratio:]\ntraining_samples = len(train_data)\nprint(train_data.shape, train_labels.shape)\nprint(valid_data.shape, valid_labels.shape)\n'

In [10]:
print(images.shape, label_one_hot.shape, valid_images.shape, valid_label_one_hot.shape)

(24750, 224, 224, 3) (24750, 10) (4949, 224, 224, 3) (4949, 10)


In [8]:
shuffle(images, label_one_hot)

# Simple Keras CNN
We'll do this twice: once with a Sequential() model, and again with a Functional() model
* 2+ Conv layers
* 2 pooling
* 2x 128-node Fully-Connected Layer
* 10-node output layer
* Batch Norm before all activations

In [10]:
#model_filename = 'keras_models/first_submission.h5'
model_filename = 'keras_models/first_resnet.h5'

In [11]:
def std(inputs):
    return 1 / math.sqrt(inputs)

In [12]:
if os.path.isfile(model_filename):
    print('Loading previously-created model,', model_filename)
    model = keras.models.load_model(model_filename)
else:
    model = keras.models.Sequential()
    # Convolution & Pooling Layers
    model.add(keras.layers.Conv2D(filters=64, kernel_size=7, strides=2, padding='same', input_shape=(224, 224, 3), kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(224*224*3))))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPooling2D())  # defaults to 2x2
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(5*5*32))))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(5*5*32))))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(5*5*32))))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPooling2D())  # defaults to 2x2
    # Fully-connected Layers
    model.add(keras.layers.Flatten())
    flatten_shape = model.layers[-1].output_shape[1]
    model.add(keras.layers.Dense(128, kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(128 * flatten_shape))))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Dense(128, kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(128*128))))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
              optimizer='adam', #keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

Loading previously-created model, keras_models/first_resnet.h5


In [13]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [28]:
batch_size = 50
batches=1000
training_samples = len(train_data)
index = 0
for i in range(batches):
        
    # Shuffle the data
    if (index + 2 * batch_size) > training_samples :  # Just ignore the partial batch, whateverrrr
        print('Shuffling training data')
        shuffle(train_data, train_labels)
        index = 0
    else:
        index += batch_size
    batch = (train_data[index:index+batch_size], train_labels[index:index+batch_size])
    model.train_on_batch(batch[0], batch[1])

Shuffling training data
Shuffling training data


In [15]:
# Need to shuffle once before training so the validation split will get a random sample
shuffle(images, label_one_hot, labels)

# Train the model

In [14]:
#loss_and_metrics = model.evaluate(train_data, train_labels, batch_size=50)
#model.fit(train_data, train_labels, epochs=5, batch_size=50)

''' Crop & Train Idea:
To get validation data:
Step through all images
    center-crop 55 random images from each folder
results in 4950 images in validation set -> 750MB

1. Load 10000 (first or second half) images, which will take about 8GB of memory 
2. Center crop 4950 images (55 images from each folder for validation set)
3. Random crop 20000 images from data (2 per image)
4. Train for an epoch
5. Repeat steps 3 & 4 for a few epochs
6. Repeat from beginning, using the other half of images
'''

def train(model, data_path='data-512', load_ratio=0.25, validation_data=None, epochs=1, batch_size=16, crops_per_image=4, bonus_training=False):
    
    if validation_data is not None:
        valid_images = validation_data[0]
        valid_label_one_hot = tf.keras.utils.to_categorical(validation_data[1], num_classes=10)
    else:
        valid_images = None
        valid_label_one_hot = None
        
    for i in range(epochs):
        images, labels = load(path=data_path, load_ratio=load_ratio)
        train_images = crop_224(images, crops_per_image=crops_per_image)
        for i in range(1, crops_per_image):
            train_images = np.concatenate((train_images, crop_224(images, crops_per_image=crops_per_image)))
        labels = np.reshape(np.tile(labels[:,0], crops_per_image), (crops_per_image * len(labels), 1))
        # Need to elementwise duplicate labels for crops_per_image > 1
        train_labels = tf.keras.utils.to_categorical(labels, num_classes=10)
        #model.fit(images, label_one_hot, epochs=5, batch_size=50, validation_split=0.2)
        # using initial_epoch does not work for some reason. May work for functional models
        print(train_images.shape, train_labels.shape)
        print(valid_images.shape, valid_label_one_hot.shape)
        model.fit(train_images, train_labels, epochs=1, batch_size=batch_size, validation_data=(valid_images, valid_label_one_hot), verbose=1)
        # get some more mileage outta training before reloading
        if bonus_training:
            shuffle(train_images, train_labels)
            model.fit(train_images, train_labels, epochs=1, batch_size=batch_size, validation_data=(valid_images, valid_label_one_hot), verbose=1)
        
        del images
        del labels
        del train_images
        del train_labels

In [10]:
valid_label_one_hot = tf.keras.utils.to_categorical(valid_labels, num_classes=10)
train(model, validation_data=(valid_images, valid_labels))

Loading images
(24480, 224, 224, 3) (24480, 10)
(4949, 224, 224, 3) (4949, 10)
Train on 24480 samples, validate on 4949 samples
Epoch 1/1
24480/24480 [==============================] - 54s 2ms/step - loss: 0.4869 - acc: 0.8312 - val_loss: 1.1828 - val_acc: 0.6937
Train on 24480 samples, validate on 4949 samples
Epoch 1/1
24480/24480 [==============================] - 50s 2ms/step - loss: 0.3214 - acc: 0.8904 - val_loss: 0.6773 - val_acc: 0.7945


In [11]:
model.save(model_filename)

# Make Predictions

In [10]:
submission_filename = 'submissions/fourth_submission.csv'

In [86]:
'''
Guide to making predictions:
open file
write header
For each test image:
    load the image
    crop the image in multiple plocations
    evaluate the image using the trained model and select label with highest confidence from all crops
    write filename and category name to file
save file
close file
submit file
'''
samples = 9
test_dir = 'data/test'
result = ['fname,camera']
for filename in os.listdir(test_dir):
    image = cv2.imread(os.path.join(test_dir, filename))
    test_images = np.expand_dims(center_crop(image), axis=0)
    for i in range(samples):
        test_images = np.concatenate((test_images, crop_224([image])))
    predictions = model.predict(test_images, batch_size=test_images.shape[0], verbose=0)
    category = np.argmax(predictions) % 10
    camera_name = CameraLabel(category).name.replace('_','-')
    result += [filename + ',' + camera_name]
    #print(result)
    del image
    del test_images
result = np.asarray(result)

np.savetxt(submission_filename, result, fmt="%s")

(2641,)


# Alternative Predictions (single crop)

In [13]:
# Pick the highest confidence
test_dir = 'data/test'
result = ['fname,camera']
for filename in os.listdir(test_dir):
    image = cv2.imread(os.path.join(test_dir, filename))
    test_image = np.expand_dims(np.asarray(center_crop(image)), axis=0)
    
    # I should batch this, but whatever
    category = np.argmax(model.predict(test_image, batch_size=1, verbose=0))
    camera_name = CameraLabel(category).name.replace('_','-')
    result += [filename + ',' + camera_name]
    #print(result)
    del image
    del test_image
result = np.asarray(result)

np.savetxt(submission_filename, result, fmt="%s")

# Functional Model
Ok, so we're going to build a functional model and then try to build a resnet 

In [15]:
'''
The bottleneck block of a resnet has the following structure:
    1x1 conv on the input features (no bias, no pad, 1 stride), batch norm w/ scaling, and followed by ReLU activation
    3x3 conv w/ no bias, 1 pad, 1 stride, batch norm w/ scaling, ReLU follows
    1x1 conv w/ 4x input features (no bias, no pad, 1 stride), batch norm w/ scaling, NO ACTIVATION
'''
def bottlenet_block(input_net, input_filters=64, down_sampling=False, feature_size=56, middle_filters_divisor=2, output_filters_scale=1):
    
    first_stride = 2 if down_sampling else 1
    # if I want to initialize weights, I need to know incoming size, figure that out
    
    # First Convolution
    block = keras.layers.Conv2D(filters=input_filters, kernel_size=1, strides=first_stride, use_bias=False, kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(input_filters * feature_size ** 2)))(input_net)
    block = keras.layers.BatchNormalization()(block)
    block = keras.layers.Activation('relu')(block)
    # Second Convolution
    block = keras.layers.Conv2D(filters=input_filters // middle_filters_divisor, kernel_size=3, padding='same', strides=1, use_bias=False, kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(input_filters // middle_filters_divisor * (feature_size / first_stride) ** 2)))(block)
    block = keras.layers.BatchNormalization()(block)
    block = keras.layers.Activation('relu')(block)
    # Final Convolution (no activation)
    block = keras.layers.Conv2D(filters=input_filters * output_filters_scale, kernel_size=1, strides=1, use_bias=False, kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(input_filters * output_filters_scale * (feature_size / first_stride) ** 2)))(block)
    block = keras.layers.BatchNormalization()(block)
    
    return block

In [23]:
del net

In [12]:
inputs = keras.layers.Input(shape=(224, 224, 3))

net = keras.layers.Conv2D(filters=64, kernel_size=7, strides=2, padding='same', input_shape=(224, 224, 3), kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(224*224*3)))(inputs)
net = keras.layers.BatchNormalization()(net)
net = keras.layers.Activation('relu')(net)
net = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(net)


# First Set of Bottlenecks

bottle_1 = bottlenet_block(net, input_filters=64, feature_size=56, middle_filters_divisor=1, output_filters_scale=4)
# Projection Function
net = keras.layers.Conv2D(filters=256, kernel_size=1, use_bias=False, strides=1, kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(56*56*256)))(net)
net = keras.layers.BatchNormalization()(net)
# Merge Bottleneck and Projection
net = keras.layers.Add()([bottle_1, net])
net = keras.layers.Activation('relu')(net)


bottle_2 = bottlenet_block(net, input_filters=256, feature_size=56, middle_filters_divisor=4, output_filters_scale=1)
net = keras.layers.Add()([bottle_2, net])
net = keras.layers.Activation('relu')(net)

bottle_3 = bottlenet_block(net, input_filters=256, feature_size=56, middle_filters_divisor=4, output_filters_scale=1)
net = keras.layers.Add()([bottle_3, net])
net = keras.layers.Activation('relu')(net)

# Second Set of Bottlenecks
bottle_4 = bottlenet_block(net, input_filters=256, down_sampling=True, feature_size=56, middle_filters_divisor=2, output_filters_scale=2)
net = keras.layers.Conv2D(filters=512, kernel_size=1, use_bias=False, strides=2, kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(28*28*512)))(net)
net = keras.layers.BatchNormalization()(net)
# Merge Bottleneck and Projection
net = keras.layers.Add()([bottle_4, net])
net = keras.layers.Activation('relu')(net)


bottle_5 = bottlenet_block(net, input_filters=512, feature_size=28, middle_filters_divisor=4, output_filters_scale=1)
net = keras.layers.Add()([bottle_5, net])
net = keras.layers.Activation('relu')(net)

bottle_6 = bottlenet_block(net, input_filters=512, feature_size=28, middle_filters_divisor=4, output_filters_scale=1)
net = keras.layers.Add()([bottle_6, net])
net = keras.layers.Activation('relu')(net)

bottle_7 = bottlenet_block(net, input_filters=512, feature_size=28, middle_filters_divisor=4, output_filters_scale=1)
net = keras.layers.Add()([bottle_7, net])
net = keras.layers.Activation('relu')(net)

# Third Set of Bottlenecks
bottle_8 = bottlenet_block(net, input_filters=512, down_sampling=True, feature_size=28, middle_filters_divisor=2, output_filters_scale=2)
net = keras.layers.Conv2D(filters=1024, kernel_size=1, use_bias=False, strides=2, kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(14*14*1024)))(net)
net = keras.layers.BatchNormalization()(net)
# Merge Bottleneck and Projection
net = keras.layers.Add()([bottle_8, net])
net = keras.layers.Activation('relu')(net)

bottle_9 = bottlenet_block(net, input_filters=1024, feature_size=14, middle_filters_divisor=4, output_filters_scale=1)
net = keras.layers.Add()([bottle_9, net])
net = keras.layers.Activation('relu')(net)

bottle_10 = bottlenet_block(net, input_filters=1024, feature_size=14, middle_filters_divisor=4, output_filters_scale=1)
net = keras.layers.Add()([bottle_10, net])
net = keras.layers.Activation('relu')(net)

bottle_11 = bottlenet_block(net, input_filters=1024, feature_size=14, middle_filters_divisor=4, output_filters_scale=1)
net = keras.layers.Add()([bottle_11, net])
net = keras.layers.Activation('relu')(net)

bottle_12 = bottlenet_block(net, input_filters=1024, feature_size=14, middle_filters_divisor=4, output_filters_scale=1)
net = keras.layers.Add()([bottle_12, net])
net = keras.layers.Activation('relu')(net)

bottle_13 = bottlenet_block(net, input_filters=1024, feature_size=14, middle_filters_divisor=4, output_filters_scale=1)
net = keras.layers.Add()([bottle_13, net])
net = keras.layers.Activation('relu')(net)

# Fourth Set of Bottlenecks
bottle_14 = bottlenet_block(net, input_filters=1024, down_sampling=True, feature_size=14, middle_filters_divisor=2, output_filters_scale=2)
net = keras.layers.Conv2D(filters=2048, kernel_size=1, use_bias=False, strides=2, kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(7*7*2048)))(net)
net = keras.layers.BatchNormalization()(net)
# Merge Bottleneck and Projection
net = keras.layers.Add()([bottle_14, net])
net = keras.layers.Activation('relu')(net)

bottle_15 = bottlenet_block(net, input_filters=2048, feature_size=7, middle_filters_divisor=4, output_filters_scale=1)
net = keras.layers.Add()([bottle_15, net])
net = keras.layers.Activation('relu')(net)

bottle_16 = bottlenet_block(net, input_filters=2048, feature_size=7, middle_filters_divisor=4, output_filters_scale=1)
net = keras.layers.Add()([bottle_16, net])
net = keras.layers.Activation('relu')(net)

# Final Pooling. Should have 2048 outputs
net = keras.layers.AveragePooling2D(pool_size=(7,7))(net)

# Final Fully-Connected Layers
flatten_shape = 2048
net = keras.layers.Flatten()(net)
#net = keras.layers.Dense(1000, kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(128 * flatten_shape)))(net)
#net = keras.layers.BatchNormalization()(net)
#net = keras.layers.Activation('relu')(net)

predictions = keras.layers.Dense(10, activation='softmax', kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(10 * flatten_shape)))(net)

model = keras.layers.Model(inputs=inputs, outputs=predictions)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', #keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [13]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [17]:
train(model, validation_data=(valid_images, valid_labels), epochs=1, crops_per_image=4)

Loading images


MemoryError: 

In [17]:
model.save('keras_models/first_resnet.h5')

In [19]:
# This returns a tensor
inputs = keras.layers.Input(shape=(224, 224, 3))

net = keras.layers.Conv2D(filters=64, kernel_size=7, strides=2, padding='same', input_shape=(224, 224, 3), kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(224*224*3)))(inputs)
net = keras.layers.BatchNormalization()(net)
net = keras.layers.Activation('relu')(net)
net = keras.layers.MaxPooling2D()(net)  # defaults to 2x2
net = keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(5*5*32)))(net)
net = keras.layers.BatchNormalization()(net)
net = keras.layers.Activation('relu')(net)
net = keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(5*5*32)))(net)
net = keras.layers.BatchNormalization()(net)
net = keras.layers.Activation('relu')(net)
net = keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(5*5*32)))(net)
net = keras.layers.BatchNormalization()(net)
net = keras.layers.Activation('relu')(net)
net = keras.layers.MaxPooling2D()(net)  # defaults to 2x2
# Fully-connected Layers
net = keras.layers.Flatten()(net)
#flatten_shape = net.layers[-1].output_shape[1]
flatten_shape = 50176  # currently bused
net = keras.layers.Dense(128, kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(128 * flatten_shape)))(net)
net = keras.layers.BatchNormalization()(net)
net = keras.layers.Activation('relu')(net)
net = keras.layers.Dense(128, kernel_initializer=keras.initializers.TruncatedNormal(stddev=std(128*128)))(net)
net = keras.layers.BatchNormalization()(net)
net = keras.layers.Activation('relu')(net)
predictions = keras.layers.Dense(10, activation='softmax')(net)
                          
model = keras.layers.Model(inputs=inputs, outputs=predictions)

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 112, 112, 64)      9472      
_________________________________________________________________
batch_normalization_21 (Batc (None, 112, 112, 64)      256       
_________________________________________________________________
activation_21 (Activation)   (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 56, 56, 64)        36928     
_________________________________________________________________
batch_normalization_22 (Batc (None, 56, 56, 64)        256       
__________

In [49]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', #keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [2]:
#loss_and_metrics = model.evaluate(train_data, train_labels, batch_size=50)
#model.fit(train_data, train_labels, epochs=5, batch_size=50)

''' Crop & Train Idea:
To get validation data:
Step through all images
    center-crop 55 random images from each folder
results in 4950 images in validation set -> 750MB

1. Load 10000 (first or second half) images, which will take about 8GB of memory 
2. Center crop 4950 images (55 images from each folder for validation set)
3. Random crop 20000 images from data (2 per image)
4. Train for an epoch
5. Repeat steps 3 & 4 for a few epochs
6. Repeat from beginning, using the other half of images
'''

def train(model, data_path='data-512', load_ratio=0.25, validation_data=None, epochs=1, crops_per_image=4):
    valid_images = validation_data[0]
    valid_label_one_hot = tf.keras.utils.to_categorical(validation_data[1], num_classes=10)

    for i in range(epochs):
        images, labels = load(path=data_path, load_ratio=load_ratio)
        train_images = crop_224(images, crops_per_image=crops_per_image)
        for i in range(1, crops_per_image):
            train_images = np.concatenate((train_images, crop_224(images, crops_per_image=crops_per_image)))
        labels = np.reshape(np.tile(labels[:,0], crops_per_image), (crops_per_image * len(labels), 1))
        # Need to elementwise duplicate labels for crops_per_image > 1
        train_labels = tf.keras.utils.to_categorical(labels, num_classes=10)
        #model.fit(images, label_one_hot, epochs=5, batch_size=50, validation_split=0.2)
        # using initial_epoch does not work for some reason. May work for functional models
        model.fit(train_images, train_labels, epochs=1, batch_size=50, validation_data=(valid_images, valid_label_one_hot), verbose=1)
        # get some more mileage outta training before reloading
        shuffle(train_images, train_labels)
        model.fit(train_images, train_labels, epochs=1, batch_size=50, validation_data=(valid_images, valid_label_one_hot), verbose=1)
        del images
        del labels
        del train_images
        del train_labels
